# Revision History

In [ ]:
# Change_date         revision_number     change_description                           author
# 04/25/2024          1                   initial check-in                             Kranthi
# 05/06/2024          2                   removed file count check                     Kranthi

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType,DateType
spark.conf.set("spark.sql.sources.partitionOverwriteMode","DYNAMIC")
from pyspark.sql.window import *
from dateutil import tz
from datetime import datetime
#from pyspark.sql.functions import row_number

In [ ]:
%run /utils/common_functions

In [ ]:
%run /utils/merge_data_notebook

# define file schema

In [ ]:
schema = StructType([
    StructField("Warehouse", StringType(), True),
    StructField("Plant", StringType(), True),
    StructField("PoNumber", StringType(), True),
    StructField("PoLine", StringType(), True),
    StructField("Material", StringType(), True),
    StructField("Size", StringType(), True),
    StructField("Width", StringType(), True),
    StructField("PackageBarCode", StringType(), True),
    StructField("ReferenceCaseNumber", StringType(), True),     
    StructField("DateReceived", StringType(), True),
    StructField("CartonNumber", StringType(), True),
    StructField("PickControlNumber", StringType(), True),
    StructField("ShipDate", StringType(), True),
    
])

# Validate the number of files and date

In [ ]:
from datetime import datetime

class InvalidInputError(Exception):
    pass

file_cnt = 0
file_dates = []
today_dt = datetime.today().strftime('%Y-%m-%d')
print("today_dt::",today_dt)
for j in mssparkutils.fs.ls(f'{raw_adls_path}AS400/BrandProtection/WM/'):
  if j.size>0:  ## ignore archive folder
    #file_dates.append(datetime.strftime(datetime.strptime(j.name.split('_')[0],'%Y%m%d'),'%Y-%m-%d'))
    file_cnt = file_cnt+1
#print('cnt::',file_cnt, 'file_date::',file_dates)
print('no of files cnt::',file_cnt)
#print("set to string date::",''.join(set(file_dates))) ## convert set to string
# try: 
#     #if (file_cnt == 5 and ''.join(set(file_dates)) ==  today_dt):
#     if (file_cnt == 5):
#         print('count is 5 and all dates belong to Today - continue processing')
#     else:
#         raise InvalidInputError("Incorrect date or # of files") 
# except Exception as e:
#     print("Error::", str(e))
#     raise           

# Move the data to Gold layer - original code

In [ ]:
from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/Chicago')
dt_time = (datetime.now()).strftime('%Y-%m-%dT%H:%M:%SZ')

utc = datetime.strptime(dt_time, "%Y-%m-%dT%H:%M:%SZ")
print("utc::",utc)
utc = utc.replace(tzinfo=from_zone)
cst = utc.astimezone(to_zone)
print("utc2::",utc)
print("cst1::",cst)
print("cst2::",cst.strftime('%H:%M:%S').replace(':',''))
current_cst_time = cst.strftime('%H:%M:%S').replace(':','')
print("current_cst_time",current_cst_time)

df_raw = spark.read.format('csv')\
         .option("header", "false")\
         .schema(schema)\
         .load(f"{raw_adls_path}AS400/BrandProtection/WM/")
df = df_raw.selectExpr('Warehouse'
,'Plant'
,'PoNumber'
,'PoLine'
,'Material'
,'Size'
,'Width'
,'PackageBarCode'
,'ReferenceCaseNumber'
,'CartonNumber'
,"date_format(to_date(DateReceived,'yyyyMMdd'),'yyyy-MM-dd') as DateReceived"
,'PickControlNumber'
,"date_format(to_date(ShipDate,'yyyyMMdd'),'yyyy-MM-dd') as ShipDate")
display(df)
if df.isEmpty():
  print('dataframe is empty')
else:
  print('dataframe is not empty')  
  dict_tables = {
        "AS400/BrandProtection/WM": { "source_df": df
                  ,"where_condition":"""target.CartonNumber = source.CartonNumber"""                        
                  ,"target_table":"lakedb_gold.brand_protection_wm_data"  
                  ,"partition":""               
                  ,"path": gold_adls_path + 'AS400/BrandProtection/WM'
                  }
                }
  perform_merge(dict_tables)                   
    
#if file_cnt ==5:
for j in mssparkutils.fs.ls(f"{raw_adls_path}AS400/BrandProtection/WM/"):
  if j.size>0:  
    print(f'moving ', j.name, ' to archive' )
    file_name = j.name.replace('.csv','') 
    mssparkutils.fs.mv(f"{raw_adls_path}AS400/BrandProtection/WM/{j.name}", f"{raw_adls_path}AS400/BrandProtection/WM/archive/{file_name}_{current_cst_time}.csv",overwrite=True)    


# Validate the end result

In [ ]:
%%sql
--drop table lakedb_gold.brand_protection_wm_data;

select * from lakedb_gold.brand_protection_wm_data;
-- where snapshotdate = date_format(CURRENT_DATE,'yyyy-MM-dd')

# Check the raw data

In [ ]:
spark.sql(f"create table if not exists raw.brand_protection_wm_raw USING CSV LOCATION '{raw_adls_path}AS400/BrandProtection/WM/archive'") 


In [ ]:
%%sql
select * from raw.brand_protection_wm_raw;